# Differential Privacy demo in Scala
This notebook shows how Scala developers can use differential privacy algorithms available in libdifferentialprivacy. See notebooks/README.md if you can't make it run.

The content of this notebook is based on [Animals and Carrots](https://github.com/google/differential-privacy/blob/e819e03a20f9d7b0a30f2547c00ba74065b3f549/examples/cc/report_the_carrots.cc) from Google's differential-privacy library.

This takes care of dependencies, classpath and imports:

In [1]:
%classpath add mvn com.google.protobuf protobuf-java 3.11.4
%classpath add mvn org.apache.commons commons-math3 3.6.1
%classpath add mvn com.google.guava guava 28.2-jre
%classpath add mvn com.github.tototoshi scala-csv 1.3.6

// Download a jar from a server (temporary solution before release)
%classpath add mvn commons-io commons-io 2.6
import org.apache.commons.io.FileUtils
import java.io.File
import java.net.URL

val dpUrl = "http://replomancer.net/OpenMined/libdifferentialprivacy-1.0.jar"
val dpFile = System.getProperty("java.io.tmpdir") ++ "/differentialprivacy-1.0.jar"
FileUtils.copyURLToFile(new URL(dpUrl), new File(dpFile))
%classpath add dynamic dpFile


import java.io._
import collection.JavaConverters._
import com.github.tototoshi.csv._

import com.google.privacy.differentialprivacy._

scala.collection.JavaConverters$@dc9aaf7

First we load the carrots data from a CSV file:

In [2]:
val reader = CSVReader.open(new File("../example_data/carrots_demo/animals_and_carrots.csv"))

// read numbers only (second column) and convert them to Double:
val carrotsScalaData = reader.all().map(row => row(1).toDouble)

reader.close()

// convert Scala types to Java types:
val carrotsData = carrotsScalaData.map{ i:Double => i:java.lang.Double }.asJava


[1.0, 88.0, 35.0, 99.0, 69.0, 14.0, 77.0, 53.0, 94.0, 67.0, 92.0, 87.0, 70.0, 31.0, 14.0, 14.0, 61.0, 57.0, 68.0, 13.0, 21.0, 38.0, 92.0, 39.0, 46.0, 36.0, 23.0, 76.0, 8.0, 69.0, 35.0, 83.0, 40.0, 74.0, 17.0, 77.0, 52.0, 31.0, 14.0, 40.0, 46.0, 99.0, 44.0, 15.0, 89.0, 36.0, 98.0, 20.0, 56.0, 90.0, 5.0, 75.0, 56.0, 23.0, 49.0, 83.0, 55.0, 22.0, 7.0, 16.0, 91.0, 80.0, 21.0, 56.0, 10.0, 28.0, 29.0, 19.0, 73.0, 45.0, 5.0, 20.0, 28.0, 45.0, 39.0, 64.0, 22.0, 7.0, 30.0, 10.0, 48.0, 60.0, 73.0, 82.0, 96.0, 82.0, 38.0, 84.0, 39.0, 12.0, 75.0, 75.0, 45.0, 87.0, 91.0, 33.0, 40.0, 0.0, 67.0, 63.0, 16.0, 93.0, 19.0, 72.0, 46.0, 73.0, 98.0, 86.0, 3.0, 64.0, 94.0, 75.0, 2.0, 87.0, 74.0, 79.0, 56.0, 51.0, 77.0, 81.0, 42.0, 90.0, 96.0, 4.0, 58.0, 73.0, 27.0, 56.0, 80.0, 10.0, 35.0, 86.0, 100.0, 16.0, 7.0, 30.0, 84.0, 50.0, 86.0, 21.0, 15.0, 66.0, 75.0, 71.0, 56.0, 52.0, 99.0, 45.0, 84.0, 99.0, 51.0, 37.0, 96.0, 90.0, 92.0, 80.0, 96.0, 31.0, 39.0, 2.0, 68.0, 53.0, 47.0, 82.0, 51.0, 57.0, 10.0, 28.0, 91

# Farmer Fred

It is a new day. Farmer Fred is ready to ask the animals about their carrot consumption.

Here's the initial value of privacy budget (epsilon) with some other settings:

In [3]:
var privacyBudget = 4.0  // we set a very high value here to improve accuracy on the small example dataset

val queryEpsilon = 1.0  // default amount of privacy budget we use per query

// Unlike the C++ library the Java version currently does not support privately inferred bounds so these have to be set manually
val lowerBound = 0.0
val upperBound = 100.0

val maxPartitions = 1
val maxContributions = 1

1

Farmer Fred asks the animals how many total carrots they have eaten. The animals know the true sum but report the differentially private sum to Farmer Fred. But first, they ensure that Farmer Fred still has privacy budget left.

In [4]:
print(f"\nPrivacy budget remaining: $privacyBudget%.2f\n")
val trueSum = carrotsScalaData.sum
print(f"True sum: $trueSum%.2f\n")

if (privacyBudget >= queryEpsilon) {
    privacyBudget -= queryEpsilon
    val privateSum = BoundedSum.builder()
                               .lower(lowerBound)
                               .upper(upperBound)
                               .maxPartitionsContributed(maxPartitions)
                               .epsilon(queryEpsilon)
                               .build()
    privateSum.addEntries(carrotsData)
    val sumResult = privateSum.computeResult()
    print(f"DP sum: $sumResult%.2f\n")
} else
    print("Not enough privacy budget left!")


Privacy budget remaining: 4.00
True sum: 9649.00
DP sum: 9630.66


null

Farmer Fred catches on that the animals are giving him DP results.

He asks for the mean number of carrots eaten, (TODO: "but this time, he wants some additional accuracy information to build his intuition." requires features yet unavailable in Java).

In [5]:
print(f"\nPrivacy budget remaining: $privacyBudget%.2f\n")
val trueMean = trueSum / carrotsData.size
print(f"True mean: $trueMean%.2f\n")

if (privacyBudget >= queryEpsilon) {
    privacyBudget -= queryEpsilon
    val privateMean = BoundedMean.builder()
                                 .lower(lowerBound)
                                 .upper(upperBound)
                                 .maxPartitionsContributed(maxPartitions)
                                 .maxContributionsPerPartition(maxContributions)
                                 .epsilon(queryEpsilon)
                                 .build()
    privateMean.addEntries(carrotsData)
    val meanResult = privateMean.computeResult()
    print(f"DP mean: $meanResult%.2f\n")
} else
    print("Not enough privacy budget left!")


Privacy budget remaining: 3.00
True mean: 53.02
DP mean: 53.12


null

Fred wonders how many gluttons are in his zoo. How many animals ate over 90 carrots?

In [7]:
print(f"\nPrivacy budget remaining: $privacyBudget%.2f\n")
val trueCount = carrotsScalaData.filter(_ > 90).size
print(f"True count: $trueCount%d\n")

if (privacyBudget >= queryEpsilon) {
    privacyBudget -= queryEpsilon
    val privateCount = Count.builder()
                            .maxPartitionsContributed(maxPartitions)
                            .epsilon(queryEpsilon)
                            .build()
    privateCount.incrementBy(trueCount)
    val countResult = privateCount.computeResult()
    print(f"DP count: $countResult%d\n")
} else
    print("Not enough privacy budget left!")



Privacy budget remaining: 2.00
True count: 21
DP count: 20


null

If you rerun one of the previous cells a couple of times you will eventually see the privacy budget running out.